In [1]:
import rioxarray as rxr
import os
from rasterio.plot import show
from PIL import Image
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import geopandas as gpd
import rasterio.features
from shapely.geometry import shape
import pandas as pd
import time

In [2]:
# retreive some infos on fully contained segments
contained_file_50 = 'data/contained_segments_50.txt'
contained_file_0001 = 'data/contained_segments_0001.txt'
contained_file_100 = 'data/contained_segments_100.txt'

In [3]:
folder = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/normalized_output"

list_bands = ['02', '03', '04', '05', '06', '07', '08', '8A', '11', '12']

Perform 7310

- Satellite raster data (Sentinel 2)
	- 10 spectral bands, one per file (s2_2020_B02.tif, s2_2020_B03.tif, etc...)
	- Each file s2_2020_BXX.tif: contains an array of size (73, 4513, 5183)=(nb. dates, height, width).

In [9]:
start = 1 # always 1
acquisition = 73 # 73
def get_7310(folder, pixel):
    x = pixel[0]
    y = pixel[1]
    pixel_7310 = []
    for band in list_bands:
        #print(band)
        for period in range(start, acquisition+1):
            #print(period)
            band_file = 's2_2020_B' + band + '.tif'
            acquisition_file = os.path.join(folder, f'normalized_{band_file}_Acquisition_{period}.tif')
            with rasterio.open(acquisition_file) as src:
                value = src.read(1)[x, y]
            pixel_7310.append(value)

    return '[' + ','.join(map(str, pixel_7310)) + ']'

In [10]:
# test the function for a pixel
pixel = [100, 100]
# compute the time taken
start_time = time.time()
pixel_7310 = get_7310(folder, pixel)
end_time = time.time()
print(f"Time taken: {end_time - start_time} seconds")

Time taken: 9.843824625015259 seconds


In [ ]:
# take random pixels
number_of_pixels = 2
pixels = []
for i in range(number_of_pixels = 2):
    x = np.random.randint(0, 3000)
    y = np.random.randint(0, 3000)
    pixels.append([x, y])

start_time = time.time()
# get the 7310 values for each pixel
pixels_7310 = []
for pixel in pixels:
    pixels_7310.append(get_7310(folder, pixel))
end_time = time.time()
print(f'Elapsed time for {number_of_pixels} pixels: {end_time - start_time} seconds')

Elapsed time: 805.9083890914917


In [ ]:
# show type of pixel_7310
print((pixel_7310))

[0.5287162,0.40724948]


Iterate through each pixel of each segment, and for each pixel, perform 7310 conversion.

In [ ]:
def load_data_from_file(contained_file):
    # load the data
    with open(contained_file) as f:
        lines = f.readlines()
        # reteive line infos
        segment_id = []
        polygon_id = []
        class_id = []
        pixels = []
        for line in lines:
            # retrieve segment id
            segment_id.append(line.split(',')[0].split(':')[1].strip())
            # retrieve polygon id
            polygon_id.append(line.split(',')[1].split(':')[1].strip())
            # retrieve class id
            class_id.append(line.split(',')[2].split(':')[1].strip())
            # retrieve pixels
            pixel_list_str = line.split(':')[4]
            pixel_list_str = pixel_list_str[3:-3]
            pixel_list = list(pixel_list_str.split('], ['))
            pixel_list = [pixel.split(', ') for pixel in pixel_list]
            pixel_list = [[int(pixel[0]), int(pixel[1])] for pixel in pixel_list]
            pixels.append(pixel_list)
        
        # create a dataframe
        df = pd.DataFrame()
        df['segment_id'] = segment_id
        df['polygon_id'] = polygon_id
        df['class_id'] = class_id
        df['pixels'] = pixels
        
        return df

In [ ]:
# retreive some infos on fully contained segments
contained_file_50 = 'data/contained_segments_50.txt'
contained_file_0001 = 'data/contained_segments_0001.txt'
contained_file_100 = 'data/contained_segments_100.txt'

In [ ]:
dataframe_segment = load_data_from_file(contained_file_50)

In [ ]:
# show mean number of pixels per segment
mean_pixels = np.mean([len(pixels) for pixels in dataframe_segment['pixels']])
print(mean_pixels)

# show for polygon 0 first segment number of pixels
print(len(dataframe_segment['pixels'][0]))

16.178727841501566
16


In [ ]:
import csv

# group by polygon ID
grouped = dataframe_segment.groupby('polygon_id')
folder_end = "D:/General/ExaplAInability_Data/transfer_6060512_files_e989f8bb/processed_Data"
iterations = len(grouped)

# for each polygon, create a file from 'folder' path named after the polygon class
for name, group in grouped:
    # show iteration
    print(f"iteration {name} / {iterations}")
    class_id = group['class_id'].iloc[0]
    file_path = os.path.join(folder_end, f'{int(float(class_id))}_{int(float(name))}.txt')

    # the polygon file contains as rows as segments contain in the polygon
    # each row is a list of the 7310 values of each pixels of the segments
    with open(file_path, 'w', newline='\n') as f:
        for index, row in group.iterrows():
            pixel_rows = []
            for pixel in row['pixels']:
                pixel_7310 = get_7310(folder, pixel)
                pixel_rows.append((pixel_7310))
            f.write(''.join(pixel_rows) + '\n')
